In [1]:
import csv
recipe_url_list = []
with open("output/recipe_url_list.csv","r",encoding="utf-8") as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        recipe_url_list.append((row[0],row[1]))
# recipe_url_list

In [2]:
# len(recipe_url_list)

In [3]:
subcats = []
for subcat in recipe_url_list:
    subcats.append(subcat[0])
subcat_filter = set(subcats)

In [ ]:
# 為了保持順序性做的處理
subcat_set = []
for subcat in subcats:
    if subcat in subcat_filter:
        if subcat_set.count(subcat) == 0:
            subcat_set.append(subcat)
# len(subcat_set)
# subcat_set

In [5]:
r = range(0, len(subcat_set), 10)
groups = []
for g in r:
    groups.append(g)
groups[-1] = len(subcat_set)
# groups

In [6]:
devided_subcat_filter = []
for i in range(len(groups)-1):
    devided_subcat_filter.append(subcat_set[groups[i]:groups[i+1]])

In [7]:
# 測試
# for dsf in devided_subcat_filter:
#     print(dsf)
# len(devided_subcat_filter)

In [8]:
# 先把所有的 recipe 塞進一個 queue
import queue
recipe_queue = queue.Queue()
for recipe in recipe_url_list:
    recipe_queue.put(recipe)
# recipe_queue.get()

In [9]:
# recipe_queue.qsize()

In [10]:
# 產生 Queue 的 list
divided_recipe_url_list = []

tmp = None # 暫存用來當分界點比對的那份食譜
for dsf in devided_subcat_filter:
#     print(dsf)
    ten_subcats_recipes = []
    while recipe_queue.qsize() > 0:
        recipe = recipe_queue.get()
        if recipe[0] in dsf:
            if tmp is not None:
                ten_subcats_recipes.append(tmp) # 把分界點的那份食譜放進去 -> 該分類食譜list的第一個元素
                tmp = None
            ten_subcats_recipes.append(recipe)
        else:
            tmp = recipe
            divided_recipe_url_list.append(ten_subcats_recipes)
            break
    # 最後一個分類食譜的list
    if recipe_queue.qsize() == 0:
        divided_recipe_url_list.append(ten_subcats_recipes)
        print("塞完了")

塞完了


In [11]:
# 每十個分類的食譜塞進一個Queue
import queue
queue_list = []
for drul in divided_recipe_url_list:
    recipe_queue = queue.Queue()
    for each_recipe in drul:
        recipe_queue.put(each_recipe)
    queue_list.append(recipe_queue)

In [12]:
# # 測試
# print(len(divided_recipe_url_list))
# sum = 0
# for dsf in divided_recipe_url_list:
#     sum += len(dsf)
# print(sum)
# # divided_recipe_url_list[0]

In [13]:
# # 測試 
# print(len(queue_list))
# sum = 0
# for q in queue_list:
#     sum += q.qsize()
# print(sum)
# queue_list

In [14]:
# 要把食物分類塞到不同的Queue，這樣執行緒才會快 
# 已改寫下面的邏輯

In [15]:
# 完整的Try-catch架構
# Try-catch的完整結構應該是這樣：try, except, else, finally
# try：需要被監控是否會出錯的程式區塊
# except：出了哪種錯誤，要有怎樣相對應的處理
# else：都沒錯誤，就會執行此區塊的程式
# finally：不論如何都會執行此區塊的程式

In [16]:
# *** python 3.6 版本以上，dict 會記住元素的插入順序，大部分情況下不再需要使用OrderedDict ***
import sys
import time
from random import randint


# 進入每個食譜
def getRecipeJson(recipe_to_crawl):
    
    headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
              }
    
    try:
        recipe_html = BeautifulSoup(requests.get(recipe_to_crawl[1], headers=headers).text)
#         print("get html successfully")
    except:
        # sys.exc_info()[0] 就是用來取出except的錯誤訊息的方法
        print("Unexpected error:", sys.exc_info()[0])
        print("failed to download %s" %(str(recipe_to_crawl)))
        
        return recipe_to_crawl
    
    else:
        # 檢查是否被伺服器ban了
        if recipe_html.find('h1') is not None:
            # 取得食譜名、食譜圖片網址、材料清單
            # recipe_name : str
            recipe_name = recipe_html.find("h1").text.strip()

            # img_src : str
            recipe_img_src = recipe_html.find("img", class_="main-pic")["src"]

            # ingredients : dict
            ingredients = recipe_html.find_all("div", itemprop="ingredients")
            ingredient_preparation = {}
            for ingre in ingredients:
                ingre_name = ingre.find("div", class_="ingredient-name").text
                ingre_unit = ingre.find("div", class_="ingredient-unit").text
                ingredient_preparation[ingre_name] = ingre_unit

            # recipe_catergory : str 
            recipe_category = recipe_to_crawl[0]

            #  做成一個 Recipe 物件{}
            a_recipe = {}
            a_recipe["食譜名稱"] = recipe_name
            a_recipe["料理圖片"] = recipe_img_src
            a_recipe["材料"] = ingredient_preparation
            a_recipe["食譜網址"] = recipe_to_crawl[1]
            a_recipe["食譜分類"] = recipe_category

            return a_recipe
        
        else:            
            print(recipe_html.getText()) # surver replies : Retry later
            random_int = randint(1,10)
            print("let's take a break for %s seconds" % (random_int))
            time.sleep(random_int)
            
            return recipe_to_crawl

In [17]:
import time
import threading
import queue
from random import randint

class Worker(threading.Thread):
    def __init__(self, queue, worker_num, qnum, lock, error_writer_lock):
        # 繼承父類別的方法 兩種寫法
        threading.Thread.__init__(self)
        # super(Worker,self).__init__()
        
        self.queue = queue
        self.num = worker_num
        self.qnum = qnum
        self.lock = lock
        self.error_writer_lock = error_writer_lock

    def run(self):
        while self.queue.qsize() > 0:
            
            recipe_to_crawl = self.queue.get()
            recipe_subcat_name = recipe_to_crawl[0]
            
            a_recipe = getRecipeJson(recipe_to_crawl)
            
            if type(a_recipe) is dict:
                # 取得 lock
                self.lock.acquire()
#                 print("Lock acquired by Worker %d" % self.num)

                try:
                    # 不能讓多個執行緒同時進行的工作
                    # 存進 recipes.json
                    with open("recipe_json_output/recipes_%s.json" %(recipe_subcat_name), "a", encoding="utf-8") as txt_outfile:
                        txt_outfile.write(repr(a_recipe) + ",")
#                     print("Worker %d of queue %s saved the recipe of %s to recipes_%s.json" % (self.num, self.qnum, a_recipe["食譜名稱"], recipe_subcat_name))
                    if self.queue.qsize != 0:
                        pass
#                         print("There are now %d recipes to retrieve in queue %d " % (self.queue.qsize(),self.qnum))
                    else:
                        print("The queue %d is empty" %(self.qnum))
                except IOError as e:
                    print(e.args[0],e.args[1])
                    print("%s的食譜沒有存進去" %(a_recipe["食譜名稱"]))
                finally:
                    # 釋放 lock
                    self.lock.release()
                    time.sleep(randint(1,10))
#                     print("Lock released by Worker %d" % self.num)
            elif type(a_recipe) is tuple:
                # 把沒抓成功的食譜存起來
                self.error_writer_lock.acquire()
                with open("recipes_crawl_failure.json", "a", encoding="utf-8") as failFile:
                    a_recipe_str = repr(recipe_to_crawl)
                    failFile.write(a_recipe_str + ",")
                self.error_writer_lock.release()

In [18]:
# 1、執行緒共享建立它的程序的地址空間,程序有自己的地址空間

# 2、執行緒可以訪問程序所有的資料，執行緒可以相互訪問

# 3、執行緒之間的資料是獨立的

# 4、子程序複製執行緒的資料

# 5、子程序啟動後是獨立的 ，父程序只能殺掉子程序，而不能進行資料交換

# 6、修改執行緒中的資料，都是會影響其他的執行緒，而對於程序的更改，不會影響子程序

In [19]:
# 主程式區
from bs4 import BeautifulSoup
from tools.countTimeDecorator import countTimeDecorator
import csv
import time
import queue
import requests
import threading
import sys
import os

# 建立 lock
# 檔案IO 安全保護鎖
# lock = threading.Lock()
# print(lock)
error_writer_lock = threading.Lock()

@countTimeDecorator(time_unit="h")
def startToCrawl():
    
    qnum = 0
    lock_list = []
    worker_threads_list = []
    for ten_subcat_queue in queue_list:
        qnum += 1
        lock = threading.Lock()
        lock_list.append(lock)
        num_worker_threads = 3
        threads = []
        for worker_num in range(num_worker_threads):
            t = Worker(ten_subcat_queue, worker_num, qnum, lock, error_writer_lock)
            print("worker ", str(worker_num), "of queue num", str(qnum), "created")
            t.start()
            threads.append(t)
            time.sleep(1)

        worker_threads_list.append(threads)

    worker_list = []
    for worker_threads in worker_threads_list:
        for worker_thread in worker_threads:
            worker_list.append(worker_thread)


    print("I am the main thread, I am waiting for the threads finishing their jobs")
    for worker in worker_list:
        worker.join()

    # with open("recipes.json", "r", encoding="utf-8") as jsonFile:
    #     dict_strs = jsonFile.read()
    #     recipes = list(eval(dict_strs))

    # with open("clean_recipes.json", "w", encoding="utf-8") as jsonFile:
    #     json_str = repr(recipes)
    #     jsonFile.write(json_str)

    print("Done.")
    
if __name__ == "__main__":
    startToCrawl()

worker  0 of queue num 1 created
worker  1 of queue num 1 created
worker  2 of queue num 1 created
worker  0 of queue num 2 created
worker  1 of queue num 2 created
worker  2 of queue num 2 created
worker  0 of queue num 3 created
worker  1 of queue num 3 created
worker  2 of queue num 3 created
worker  0 of queue num 4 created
worker  1 of queue num 4 created
worker  2 of queue num 4 created
worker  0 of queue num 5 created
worker  1 of queue num 5 created
worker  2 of queue num 5 created
worker  0 of queue num 6 created
worker  1 of queue num 6 created
worker  2 of queue num 6 created
worker  0 of queue num 7 created
worker  1 of queue num 7 created
worker  2 of queue num 7 created
worker  0 of queue num 8 created
worker  1 of queue num 8 created
worker  2 of queue num 8 created
worker  0 of queue num 9 created
worker  1 of queue num 9 created
worker  2 of queue num 9 created
worker  0 of queue num 10 created
worker  1 of queue num 10 created
worker  2 of queue num 10 created
worker 

In [1]:
# 把每一份json檔讀進來清理
import os
pwd = os.getcwd()
print(pwd)
data_dir = os.path.join(pwd, "recipe_json_output")
print(data_dir)
recipe_json_list = os.listdir(data_dir)
print(recipe_json_list)

for recipe_json in recipe_json_list:
    infile = "recipe_json_output/" + recipe_json
    if os.path.isfile(infile):
        with open(infile, "r", encoding="utf-8") as jsonFile:
            dict_strs = jsonFile.read()
            recipes = list(eval(dict_strs))

        outfile = "clean_recipe_json_output/" + recipe_json
        with open(outfile, "w", encoding="utf-8") as jsonFile:
            json_str = repr(recipes)
            jsonFile.write(json_str)
        
# result = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]
# print(result)

/Users/even/PycharmProjects/recipeCrawler
/Users/even/PycharmProjects/recipeCrawler/recipe_json_output
[]
